In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import re
import time 
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def get_html(url):
    response = requests.get(url)
    return soup(response.text)

In [3]:
fechas = pd.date_range(start="2024-01-01", end="2025-01-01", freq="D")
final_df = pd.DataFrame()
current_year = fechas[0].year  # Inicializa el primer año
start_time = time.time()

for fecha in fechas:
    try:

        url = f"https://es.weatherspark.com/h/d/21493/{fecha.year}/{fecha.month}/{fecha.day}/Tiempo-hist%C3%B3rico"
        html = get_html(url)
        time.sleep(4)
        
        tabla = html.find("table", {"class": "table table-condensed stickyHeader History-MetarReports-outer_table"})
        df = pd.read_html(str(tabla))[0]
    
        df_filtered = df[df['Hora'].str.match(r'^\d{1,2}:\d{2} (a\. m\.|p\. m\.)')].copy()
        df_filtered['Fecha'] = fecha
        
        final_df = pd.concat([final_df, df_filtered])

        if fecha.year != current_year:

            end_time = time.time()
            elapsed_time = end_time - start_time

            final_df.to_csv(f"../Datos/Weather_Spark/Cali/Raw_Weather_Spark_Cali_{current_year}.csv", index=False)
            print(f"Guardado archivo para el año {current_year}")
            print(f"Tiempo transcurrido para descargar los datos de {current_year}: {elapsed_time/60:.2f} minutos")

            final_df = pd.DataFrame()
            current_year = fecha.year
            start_time = time.time() 
    except Exception as e:
        #print(f"Error en la fecha {fecha}: {e}")
        continue

if not final_df.empty:
    end_time = time.time()
    elapsed_time = end_time - start_time
    final_df.to_csv(f"../Datos/Weather_Spark/Cali/Raw_Weather_Spark_Cali_{current_year}.csv", index=False)

Guardado archivo para el año 2024
Tiempo transcurrido para descargar los datos de 2024: 27.74 minutos


In [4]:
#final_df.to_csv(f"../Datos/Raw_Weather_Spark_{2025}.csv", index=False)
#final_df.to_excel(f"../Datos/Raw_Weather_Spark_{2025}.xlsx", index=False)